In [15]:
from gym_anytrading.envs import StocksEnv

# Stable baselines - rl stuff
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import PPO

import numpy as np
import pandas as pd

In [10]:
df = pd.read_csv('../Dataset/process data/nasdaq_dataset.csv')
df.head()

,Date,Time,Open,High,Low,Close,Volume,NumberOfTrades,BidVolume,AskVolume,...,AROON_UP,ADX,STOCH_K,STOCH_D,MACD,MACD_SIGNAL,HAMMER,INVERTED_HAMMER,ENGULFING,Target
0,2011/6/13,20:00:00,1350.54,1350.54,1350.54,1350.54,0,0,0,0,...,21.428571,43.819322,40.615298,45.541867,-7.932664,-8.560322,0,0,0,0
1,2011/6/14,12:00:00,1363.16,1370.18,1363.16,1368.53,0,0,0,0,...,100.000000,42.298061,58.060659,48.180735,-6.232078,-8.094673,0,0,0,0
2,2011/6/14,13:00:00,1368.51,1374.46,1368.06,1373.62,0,0,0,0,...,100.000000,41.340922,76.586371,58.420776,-4.422650,-7.360269,0,0,0,0
3,2011/6/14,14:00:00,1373.62,1374.23,1370.97,1372.80,0,0,0,0,...,92.857143,40.452150,93.929978,76.192336,-3.020020,-6.492219,0,0,0,0
4,2011/6/14,15:00:00,1372.79,1373.59,1371.59,1372.15,0,0,0,0,...,85.714286,39.626862,93.361517,87.959288,-1.938528,-5.581481,0,0,0,0


In [11]:
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df.drop(columns=['Time','Target'],inplace=True)
print(len(df))
df.head()

24519


,Open,High,Low,Close,Volume,NumberOfTrades,BidVolume,AskVolume,ATR,RSI,...,AROON_DOWN,AROON_UP,ADX,STOCH_K,STOCH_D,MACD,MACD_SIGNAL,HAMMER,INVERTED_HAMMER,ENGULFING
Date,,,,,,,,,,,,,,,,,,,,,
2011-06-13,1350.54,1350.54,1350.54,1350.54,0,0,0,0,5.104920,34.686033,...,71.428571,21.428571,43.819322,40.615298,45.541867,-7.932664,-8.560322,0,0,0
2011-06-14,1363.16,1370.18,1363.16,1368.53,0,0,0,0,6.143140,54.838940,...,64.285714,100.000000,42.298061,58.060659,48.180735,-6.232078,-8.094673,0,0,0
2011-06-14,1368.51,1374.46,1368.06,1373.62,0,0,0,0,6.161487,58.719937,...,57.142857,100.000000,41.340922,76.586371,58.420776,-4.422650,-7.360269,0,0,0
2011-06-14,1373.62,1374.23,1370.97,1372.80,0,0,0,0,5.954238,57.857319,...,50.000000,92.857143,40.452150,93.929978,76.192336,-3.020020,-6.492219,0,0,0
2011-06-14,1372.79,1373.59,1371.59,1372.15,0,0,0,0,5.671792,57.140743,...,42.857143,85.714286,39.626862,93.361517,87.959288,-1.938528,-5.581481,0,0,0


In [12]:
def add_new_data(env):
    start = env.frame_bound[0] - env.window_size
    end = env.frame_bound[1]
    prices = env.df.loc[:, 'Close'].to_numpy()[start:end]
    signal_features = env.df.loc[:, ['Open', 'High', 'Low', 'Volume', 'NumberOfTrades', 'BidVolume',
       'AskVolume', 'ATR', 'RSI', 'AD', 'AROON_DOWN', 'AROON_UP', 'ADX',
       'STOCH_K', 'STOCH_D', 'MACD', 'MACD_SIGNAL', 'HAMMER',
       'INVERTED_HAMMER', 'ENGULFING']].to_numpy()[start:end]
    return prices, signal_features

In [13]:
class MyEnv(StocksEnv):
    _process_data = add_new_data
    
env2 = MyEnv(df=df, window_size=30, frame_bound=(30,18000))

In [14]:
env_maker = lambda: env2
env = DummyVecEnv([env_maker])

In [19]:
# from stable_baselines3 import PPO
# import gym

# # Define and initialize the environment
# env = MyEnv(df=df, window_size=30, frame_bound=(18000,18500))
# obs = env.reset()

# # Define and initialize the model
# model = PPO('MlpPolicy', env)

# # Load the pre-trained model weights
# model.load("models/PPO_nasdaq_model_1")

# # Use the loaded model to interact with the environment
# obs = env.reset()
# done = False

# while not done:
#     action, _ = model.predict(obs)
#     obs, reward, done, _ = env.step(action)
#     env.render()

# env.close()

In [17]:
env = MyEnv(df=df, window_size=30, frame_bound=(18000,18500))
obs = env.reset()
while True: 
    obs = obs[np.newaxis, ...]
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    if done:
        print("info", info)
        break

info {'total_reward': 269.22999999999774, 'total_profit': 0.171467012597876, 'position': 1}
